In [0]:
import pandas as pd

In [21]:
path = 'BTC_USD_CBPro_TA.csv'
df = pd.read_csv(path)

df.drop(df.columns[0], axis=1, inplace=True)
df = df[::-1]
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,open,high,low,close,volume,volume_adi,volume_cmf,volume_fi,volume_em,volume_vpt,volume_nvi,volatility_atr,volatility_bbh,volatility_bbl,volatility_bbm,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kchi,volatility_kcli,volatility_dch,volatility_dcl,volatility_dchi,volatility_dcli,trend_macd,trend_macd_signal,trend_macd_diff,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_diff,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,momentum_rsi,momentum_mfi,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,others_dr,others_dlr,others_cr,close_diff,target
81209,454.23,455.00,454.23,455.00,8.231416,-67.873611,0.087358,-0.146628,-0.000684,-0.189544,33346.744316,0.462754,455.371421,453.711579,454.5415,0.0,0.0,454.634000,455.064000,454.204000,0.0,0.0,455.00,453.33,1.0,0.0,0.412153,0.471693,-0.059539,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81208,454.99,455.00,454.06,454.24,123.342630,-55.391254,0.032379,-67.941928,-0.001159,-0.197277,33291.044259,0.439119,455.506115,453.370885,454.4385,0.0,0.0,454.614333,454.994333,454.234333,0.0,0.0,454.99,452.94,0.0,0.0,0.406920,0.486577,-0.079657,454.556540,454.149620,0.953789,0.977819,0.024030,24.516188,1.122334,0.3115,3.947127,4.310799,-0.363672,454.2175,452.825,452.3975,452.10,96.0,24.0,72.0,40.752968,41.300770,23.211303,53.668747,21.649485,69.072165,-78.350515,0.887500,-0.164839,-0.164975,63.413318,-0.75,True
81207,454.87,455.00,454.79,454.99,22.894171,36.212073,0.159094,-0.379183,-0.000666,0.013198,33291.044259,0.400590,455.575556,453.205444,454.3905,0.0,0.0,454.637333,454.981333,454.293333,0.0,0.0,454.99,452.94,1.0,0.0,0.471703,0.506492,-0.034789,454.614093,454.142390,1.111984,0.891945,0.220039,24.474275,85.126734,0.3595,4.200384,4.349439,-0.149056,454.2175,452.825,452.1475,451.85,100.0,28.0,72.0,73.593550,67.659614,23.932434,63.476815,98.969072,87.035526,-1.030928,0.998676,0.026381,0.026378,63.683131,0.12,False
81206,454.76,454.87,454.65,454.87,15.498300,14.296140,0.113274,0.316427,-0.000632,0.003457,33291.044259,0.415251,455.525827,453.110173,454.3180,0.0,0.0,454.627667,454.974667,454.280667,0.0,0.0,454.96,452.94,0.0,0.0,0.471165,0.515189,-0.044024,454.545747,454.074581,1.195358,0.883946,0.311412,24.656868,70.605885,0.6420,4.244849,4.337114,-0.092265,454.1875,452.825,452.0450,451.80,56.0,32.0,24.0,70.271582,71.825196,23.492848,60.852594,86.597938,81.085463,-13.402062,0.961353,0.046188,0.046178,63.639961,0.21,True
81205,454.68,454.87,454.47,454.66,24.043210,7.363675,0.122594,-0.679135,-0.000584,0.001250,33275.674771,0.430270,455.468574,453.049426,454.2590,0.0,0.0,454.631333,454.981333,454.281333,0.0,0.0,454.96,452.94,0.0,0.0,0.475844,0.526195,-0.050351,454.486791,454.010948,1.154255,0.822695,0.331560,24.736838,56.687905,0.6410,4.308877,4.302260,0.006616,454.1725,452.825,451.9950,451.75,60.0,36.0,24.0,63.262020,76.426991,23.130385,61.562035,75.539568,77.581802,-24.460432,1.020412,-0.015394,-0.015395,63.564413,-0.07,True


In [29]:
import backtrader as bt
from datetime import datetime

class firstStrategy(bt.Strategy):

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.buy(size=100)
        else:
            if self.rsi > 70:
                self.sell(size=100)
                
                

#Variable for our starting cash
startcash = 10000

#Create an instance of cerebro
cerebro = bt.Cerebro()

#Add our strategy
cerebro.addstrategy(firstStrategy)

#Get Apple data from Yahoo Finance.
data = bt.feeds.Quandl(
    dataname='AAPL',
    fromdate = datetime(2016,1,1),
    todate = datetime(2017,1,1),
    buffered= True
    )

#Add the data to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(startcash)

# Run over everything
cerebro.run()

#Get final portfolio Value
portvalue = cerebro.broker.getvalue()
pnl = portvalue - startcash

#Print out the final result
print('Final Portfolio Value: ${}'.format(portvalue))
print('P/L: ${}'.format(pnl))

#Finally plot the end results
cerebro.plot(style='candlestick')

Final Portfolio Value: $11755.2851701423
P/L: $1755.2851701423006


ImportError: ignored

In [0]:
btc = requests.get("https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=500").json()['Data']

In [0]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
#nice trick to make plots full width
plt.rcParams['figure.figsize'] = [15,5]

In [0]:
#put into dataframe
btc_df = pd.DataFrame(btc)
bch_df = pd.DataFrame(bch)
#use pandas datetime feature to convert timestamp into a datatime object with units = seconds
btc_df['time'] = pd.to_datetime(btc_df['time'], unit='s')
bch_df['time'] = pd.to_datetime(bch_df['time'], unit='s')
#use the newly created datetime object as index
btc_df.set_index('time', inplace=True)
bch_df.set_index('time', inplace=True)
#rename 'close' for each instrument so they have unique names
btc_df.rename({'close':'btc'}, axis=1, inplace=True)
bch_df.rename({'close':'bch'}, axis=1, inplace=True)
#select our desired stating data
btc_df = btc_df.loc['2017-12-12':]
bch_df = bch_df.loc['2017-12-12':]

In [0]:
#Portfolio class handles trade logic
class Portfolio:
    def __init__(self):
        #self.orders = pd.DataFrame(columns=['TS','Order','tick1','tick2'])
        self._port = pd.df(columns=['time','close','target'])
#        self.current_budget = 1000000
        self.signal = None
        self.prev = None
        #bought / sold
        self.current_pos= "empty"
        #self.pnl = pd.DataFrame(columns = ['pnl'])
        self.bought_sold_price = 0
        self.stamp = 0
        #self.sold_value = 0
        #self.bot_value = 0
        self.sell_units = 0
        self.buy_units = 0
        self.value_2 = 0
        self.value_1 = 0
        self.rpl = 0
    
    def close_out(self):
        self.rpl += (1000 - self.value) + (self.value_1 - 1000)
        self.current_pos ='empty'
print("close out position")
        
          
    def position(self,ts,tick1,tick2,price,tot_trade_amount=2000):
        print()
        print(self.stamp)
        print('current pos:',self.current_pos)
        print("bought / sold price: ",self.bought_sold_price)
        print('this is prev:', self.prev)
        print('this is the signal:',self.signal)
        single_trade_amount = tot_trade_amount/2
        action = None
        
#logic for Hold signal        
        
        if self.signal =="Hold":
            
            if self.current_pos =='sold':
                print("sold tick")
                self.value_2 = self.sell_units * tick2
                self.value_1 = self.buy_units * tick1
            elif self.current_pos == 'bought':
                self.value_2 = self.sell_units * tick1
                self.value_1 = self.buy_units * tick2
            else:
                print("Hold neither bought nor sold")
                self.value_2 = 0
                self.value_1 = 0
                
                
                
          
            print("hold 1")
            print("caputured by Hold")
            
            if self.current_pos == 'bought' and price > self.mu:
                print("hold 2")
                self.close_out()
                action = "Closed out Long"
                #self.current_pos ='empty'
                
            elif self.current_pos =='sold' and price < self.mu:
                print("hold 3")
                self.close_out()
                action = "Closed out Short"
                #self.current_pos ='empty'
            else:
                print("hold 4")
                print("""take no action -> Hold""")
                action = "Held"
        
#logic for Short signal        
        
        elif self.signal =='Short':
            
            print("caputrd by Short")
            sell_units = single_trade_amount/tick2
            buy_units = single_trade_amount/tick1
            
            if self.signal == 'Short' and  self.signal != self.prev:
                print("short 1")
                if self.current_pos == 'bought':
                    self.value_2 = self.sell_units * tick2
                    self.value_1 = self.buy_units * tick1
                    self.close_out()
                elif self.current_pos == 'empty':
                    
                    print("short 2")

                    print("Went short: sold",sell_units,"units of BTC","at a price of",tick2, "and bought",buy_units,"of b*BCH at a price of",tick1)
                    #self.sold_value = sell_units*tick2
                    #self.bot_value = buy_units*tick1
                    self.sell_units = sell_units
                    self.buy_units = buy_units
                    self.value_2 = self.sell_units * tick2
                    self.value_1 = self.buy_units * tick1
self.bought_sold_price = tick2 - tick1
                    self.current_pos = 'sold'
                    action = "Went Short!"
else:
                    print("short 5")
                    print("current pos must be already sold - check!")
                    action = "Already Short!"
                    self.value_2 = self.sell_units * tick2
                    self.value_1 = self.buy_units * tick1
else:
                print("short 6")
                print("prev signal must be Short - check!")
                action = "Already Short!"
                self.value_2 = self.sell_units * tick2
                self.value_1 = self.buy_units * tick1

            
#logic for Long signal            
        
        elif self.signal =='Long':
            
            print("captured by Long")
            sell_units = single_trade_amount/tick1
            buy_units = single_trade_amount/tick2
            
            if self.signal == 'Long' and self.signal != self.prev:
                print("long 1")
                if self.current_pos == 'sold':
                    self.value_2 = self.sell_units * tick1
                    self.value_1 = self.buy_units * tick2
                    self.close_out() 
                    action = "short => close out"
                elif self.current_pos == "empty":
                    
                    print("long 2")

                    print("Went Long: sold",sell_units,"units of b*BCH","at a price of",tick1, "and bought",buy_units,"of BTC at a price of",tick2)
                    #self.sold_value = sell_units*tick1
                    #self.bot_value = buy_units*tick2
                    self.sell_units = sell_units
                    self.buy_units = buy_units
                    self.value_2 = self.sell_units * tick1
                    self.value_1 = self.buy_units * tick2
self.bought_sold_price = tick2 - tick1
                    self.current_pos = 'bought'
                    action = "Went Long!"
                    print("should be 1000", single_trade_amount)
                    print("tot trade amount", tot_trade_amount)
else:
                    print("long 5")
                    print("current pos must be already long - check")
                    action = "Already Long!"
                    self.value_2 = self.sell_units * tick1
                    self.value_1 = self.buy_units * tick2
            else:
                print("long 6")
                print("prev signal must be long - check!")
                action = "Already Long!"
                self.value_2 = self.sell_units * tick1
                self.value_1 = self.buy_units * tick2
else:
            print("not captured 1")
            print("not captured by buy sell or hold need to fix!")
            
            
        print(self.sell_units)
        print(self.buy_units)
        print(ts)
        #print("tick1: ", tick1, "tick2: ", tick2)
#calculate unrealized pnl and finally update _port()
        urpl = (1000 - self.value_2) + (self.value_1 - 1000)
        self._port.loc[len(self._port)] = [ts,self.signal,action,self.value_2,self.value_1,urpl,self.rpl]
self.prev = self.signal
        self.stamp+=1